In [21]:
import os
import sys
spark_home = os.environ.get('SPARK_HOME', None)
sys.path.insert(0, spark_home + "/python")
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.8.2.1-src.zip'))

from pyspark import SparkContext,SparkConf
from pyspark.sql import SQLContext
from pyspark.sql.types import *

from Crypto.Cipher import AES
import random

In [4]:
conf = SparkConf().setAppName("testCSVEncode").setMaster("local[4]")
sc = SparkContext(conf = conf)
sqlContext = SQLContext(sc)

In [5]:
iv = ''.join(chr(random.randint(0x48,0x48)) for i in range(16))

key = '0123456789abcdef'

def map_decode_txt(x):

    liczbyStr = x.split(' ')
    liczby = [int(x) for x in liczbyStr]
    decryptor = AES.new(key, AES.MODE_CBC, iv)                
    recoded = decryptor.decrypt(bytes(liczby))
    recodedTxt = [ chr(r) for r in recoded]
    
    drop = 1
    n = len(recoded)
    
    print(n)
    if recoded[n-drop] == 0x00:
        drop+=1
        while(recoded[n-drop] == 0x00):
            drop+=1 
    
    rr = ''.join(recodedTxt[0:n-drop+1])
    
    return(rr)

HHHHHHHHHHHHHHHH


In [6]:
rdd = sc.textFile('testBig2.csv.enc')
rdd.count()

10001

In [7]:
rddRecoded = (rdd.map(map_decode_txt)
                 .map( lambda x: x.split(';'))
                 .cache() 
              )
colnames = rddRecoded.first()
schemaString = ' '.join(colnames)

fields = [StructField(field_name, StringType(), True) for field_name in schemaString.split()]
schema = StructType(fields)

schema = StructType(fields)

rddWithoutFirstRow = rddRecoded.zipWithIndex().filter(lambda x: x[1] > 0 ).map(lambda x: x[0])

df = sqlContext.createDataFrame(rddWithoutFirstRow, schema)

In [8]:
df.show(1)

+------------------+-------------------+----------------+------------------+-----------------+-----------------+-----------------+----------------+------------------+------------------+----------------+-------------------+------------------+------------------+---------------+------------------+-----------------+------------------+-----------------+------------------+----------------+-----------------+-----------------+------------------+------------------+------------------+-----------------+-------------------+------------------+-----------------+------------------+------------------+------------------+-----------------+----------------+-----------------+-----------------+-----------------+------------------+-------------------+-----------------+-----------------+-----------------+----------------+-----------------+-----------------+-----------------+----------------+-----------------+-----------------+-----------------+----------------+----------------+------------------+------------

In [9]:
df.printSchema()

root
 |-- var1: string (nullable = true)
 |-- var2: string (nullable = true)
 |-- var3: string (nullable = true)
 |-- var4: string (nullable = true)
 |-- var5: string (nullable = true)
 |-- var6: string (nullable = true)
 |-- var7: string (nullable = true)
 |-- var8: string (nullable = true)
 |-- var9: string (nullable = true)
 |-- var10: string (nullable = true)
 |-- var11: string (nullable = true)
 |-- var12: string (nullable = true)
 |-- var13: string (nullable = true)
 |-- var14: string (nullable = true)
 |-- var15: string (nullable = true)
 |-- var16: string (nullable = true)
 |-- var17: string (nullable = true)
 |-- var18: string (nullable = true)
 |-- var19: string (nullable = true)
 |-- var20: string (nullable = true)
 |-- var21: string (nullable = true)
 |-- var22: string (nullable = true)
 |-- var23: string (nullable = true)
 |-- var24: string (nullable = true)
 |-- var25: string (nullable = true)
 |-- var26: string (nullable = true)
 |-- var27: string (nullable = true)
 |-- 

In [10]:
df.registerTempTable("dfTab")

In [18]:
sqlContext.registerFunction( "stringLength", lambda x: len(x),IntegerType())

In [17]:
sqlContext.sql('SELECT  AVG( stringLength(var1)) FROM dfTab').show()

+-------+
|    _c0|
+-------+
|17.1791|
+-------+



In [20]:
sqlContext.sql('SELECT  AVG(var1) FROM dfTab').show()

+--------------------+
|                 _c0|
+--------------------+
|-0.00145957960384...|
+--------------------+



In [67]:
sc.stop()